## Final Experiment for RVNS

In [1]:
import sys
from os import listdir
from os.path import isfile, join
import string

import numpy as np
from plot_utils import plot_two_solutions
import matplotlib.pyplot as plt 

from ioutils import parse_input
from ioutils import parse_solutions
from utils import get_distance_matrix
from utils import get_discount_matrix
from utils import allocate_paths
from utils import get_nodes
from utils import get_total_cost
from utils import get_flow_from_paths
import cplex_utils
import graph_utils as graph_utils

from ioutils import get_comparison_table_file_name
from utils import get_latest_commit_id
from utils import get_comparison_table

import pandas as pd

In [2]:
DATASET = 'AP'
INPUT_DIRECTORY = "./data/AP/generated/"
SOLUTIONS_FILE ='/home/ubuntu/VNS_for_UMApHMP/data/AP/Solutions-UMApHMP.txt'

In [3]:
from vns import get_initial_solution_random
from vns import get_initial_solution_robust
from vns import get_initial_solution_circle

from vns import local_search_best_improvement
from vns import local_search_first_improvement


from vns import reduced_VNS

from utils import NEIGHBOURHOOD_TYPES

from ioutils import get_comparison_table_file_name
from utils import get_latest_commit_id

In [4]:
NUMBER_OF_PROBLEMS = 45
OUTPUT_FILE_NAME = get_comparison_table_file_name('final_comparison_rvns', str(NUMBER_OF_PROBLEMS), get_latest_commit_id())

In [5]:
def best_ls(solution, neighbourhood_type, use_c):
    return local_search_best_improvement(solution, neighbourhood_type, use_c)

def first_ls(solution, neighbourhood_type, use_c):
    return local_search_first_improvement(solution, neighbourhood_type, use_c)

def random_init(n, p, distances, nodes_coor):
    return get_initial_solution_random(n, p, distances, nodes_coor)

def grane_init(n, p, distances, nodes_coor):
    return get_initial_solution_robust(n, p, distances, nodes_coor)

def uglovi_init(n, p, distances, nodes_coor):
    return get_initial_solution_circle(n, p, distances, nodes_coor)


In [6]:
# ------- algorithm parameters --------
# shaking_intensity = [0, 0.5]
# initializations = [random_init, grane_init, uglovi_init]
# local_searches = [best_ls, first_ls]
# max_iters = [5]
# kmax_options = [1,2,3]

In [7]:
# ------- algorithm parameters --------
shaking_intensity = [0.5]
initializations = [grane_init]
local_searches = [best_ls, first_ls]
max_iters = [5]
kmax_options = [3]

In [8]:
methods = []
names = []

for shake_int in shaking_intensity:
    for kmax in kmax_options:
        for max_iter in max_iters:
            methods.append(lambda problem, initial_solution: reduced_VNS(problem, initial_solution=initial_solution, diversification_param=shake_int, neighbourhood_types=NEIGHBOURHOOD_TYPES[:kmax], max_iter=max_iter))
            names.append(" ".join(["RVNS", "shake="+str(shake_int), "kmax="+str(kmax), "max_iter="+str(max_iter)]))


In [9]:
len(names)

1

In [10]:
names[:5]

['RVNS shake=0.5 kmax=3 max_iter=5']

In [11]:
comparison_table = get_comparison_table(
    methods,
    names,
    DATASET, INPUT_DIRECTORY, SOLUTIONS_FILE,
    from_problem=0,
    to_problems=NUMBER_OF_PROBLEMS,
    initializations=initializations,
    display_solution=True,
    display_nan=True)
comparison_table.to_csv(OUTPUT_FILE_NAME, index_label='problem')
# comparison_table = pd.read_csv("output/basic_VNS_final_comparison_35_d42a72836976d0d8abf612afa150b58ce819c49f.csv", index_col=0)

 18%|█▊        | 7/40 [00:00<00:00, 61.30it/s]

100%|██████████| 40/40 [05:30<00:00,  8.27s/it]


In [12]:
comparison_table.head(NUMBER_OF_PROBLEMS+1)

,optimal solution,RVNS shake=0.5 kmax=3 max_iter=5 grane_init - solution,RVNS shake=0.5 kmax=3 max_iter=5 grane_init - deviation (%),RVNS shake=0.5 kmax=3 max_iter=5 grane_init - time (s)
10.2,163603.940000,163603.94000,0.000000,0.0100
10.3,131581.790000,131581.78000,0.000000,0.0100
10.4,107354.730000,107354.73000,0.000000,0.0100
10.5,86028.880000,86028.88000,0.000000,0.0100
10.8,NaN,54628.75000,NaN,0.0100
20.2,168599.790000,168599.78000,0.000000,0.0300
20.3,148048.300000,148048.29000,0.000000,0.0400
20.4,131665.430000,131665.42000,0.000000,0.0600
20.5,118934.970000,118934.97000,0.000000,0.0700
20.8,NaN,91454.82000,NaN,0.1000


In [13]:
def get_all_columns_with_similar_value(df, value):
    return df.loc[:, abs(df.iloc[-1] - value) < 0.02].columns

In [14]:
time_table = comparison_table[comparison_table.columns[1::2]]
deviation_table = comparison_table[comparison_table.columns[::2]]

In [15]:
time_table

,RVNS shake=0.5 kmax=3 max_iter=5 grane_init - solution,RVNS shake=0.5 kmax=3 max_iter=5 grane_init - time (s)
10.2,163603.94000,0.0100
10.3,131581.78000,0.0100
10.4,107354.73000,0.0100
10.5,86028.88000,0.0100
10.8,54628.75000,0.0100
20.2,168599.78000,0.0300
20.3,148048.29000,0.0400
20.4,131665.42000,0.0600
20.5,118934.97000,0.0700
20.8,91454.82000,0.1000


In [16]:
# min deviation

best_method_per_dev = deviation_table.idxmin(axis=1)[-1]
worst_method_per_dev = deviation_table.idxmax(axis=1)[-1]

best_dev_value = comparison_table[best_method_per_dev]['mean']
worst_dev_value = comparison_table[worst_method_per_dev]['mean']


best_method_per_dev, best_dev_value

('RVNS shake=0.5 kmax=3 max_iter=5 grane_init - deviation (%)',
 0.17535714285714285)

In [17]:
worst_method_per_dev, worst_dev_value

('optimal solution', 142275.45928571429)

In [18]:
# min time

best_method_per_time = time_table.idxmin(axis=1)[-1]
worst_method_per_time = time_table.idxmax(axis=1)[-1]

best_time_value = comparison_table[best_method_per_time]['mean']
worst_time_value = comparison_table[worst_method_per_time]['mean']

best_method_per_time, best_time_value

('RVNS shake=0.5 kmax=3 max_iter=5 grane_init - time (s)', 8.2635)

In [19]:
worst_method_per_time, worst_time_value

('RVNS shake=0.5 kmax=3 max_iter=5 grane_init - solution', 131736.84824999998)

In [20]:
# ------- sort all columns based on mean ----------

In [21]:
ten_best_per_time = time_table.sort_values(by='mean', axis=1).iloc[:, : 10]
ten_best_per_time

,RVNS shake=0.5 kmax=3 max_iter=5 grane_init - time (s),RVNS shake=0.5 kmax=3 max_iter=5 grane_init - solution
10.2,0.0100,163603.94000
10.3,0.0100,131581.78000
10.4,0.0100,107354.73000
10.5,0.0100,86028.88000
10.8,0.0100,54628.75000
20.2,0.0300,168599.78000
20.3,0.0400,148048.29000
20.4,0.0600,131665.42000
20.5,0.0700,118934.97000
20.8,0.1000,91454.82000


In [22]:
time_table.sort_values(by='mean', axis=1).iloc[:,:]


,RVNS shake=0.5 kmax=3 max_iter=5 grane_init - time (s),RVNS shake=0.5 kmax=3 max_iter=5 grane_init - solution
10.2,0.0100,163603.94000
10.3,0.0100,131581.78000
10.4,0.0100,107354.73000
10.5,0.0100,86028.88000
10.8,0.0100,54628.75000
20.2,0.0300,168599.78000
20.3,0.0400,148048.29000
20.4,0.0600,131665.42000
20.5,0.0700,118934.97000
20.8,0.1000,91454.82000


In [23]:
ten_best_per_time.sort_values(by='mean', axis=1).to_csv("rvns_ten_best_per_time.csv", index_label='problem')
